In [6]:
import requests
import re
import datetime as dt
import pandas as pd
import time
from functools import lru_cache
from bs4 import BeautifulSoup
import fastprogress

In [7]:
from bokeh.plotting import output_notebook, show, figure
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [59]:
pd.set_option('display.max_rows', 500)

In [8]:
@lru_cache()
def convert_date(year, month_ind, day):
    try:
        return dt.date(year, month_ind + 1, day)
    except:
        return None

In [9]:
# TODO: Pull URLS

In [10]:
month = 3
movies = set()

for year in range(2015, 2020):
    response = requests.get(f"https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/{year}/{year}/{month}/{month}/None/None/None/None?show-release-date=On&show-domestic-box-office=On&show-international-box-office=On&show-worldwide-box-office=On&view-order-by=worldwide-box-office&view-order-direction=desc")
    soup = BeautifulSoup(response.text)
    table = soup.find('table')
    for row in table.findAll("tr"):
        cell = row.find("a")
        if cell:
            title = cell.attrs['href'][7:-12]
            movies.add(title)
movies

{'1-54-(Canada)',
 '10-Cloverfield-Lane',
 '12-Golden-Ducks',
 '12-jours-(France-2017)',
 '1898-Los-ultimos-de-Filipinas-(Spain)',
 '3-Storeys-(India)',
 '311-Enlarged-to-Show-Detail-3-(2019)',
 '52-Tuesdays',
 '7-Days-in-Entebbe-(UK)',
 'Aala-Kaf-Ifrit-(Tunisia)',
 'About-Scout',
 'Accidental-Love',
 'Acht-Stunden-sind-kein-Tag-(Germany)',
 'Acrimony',
 'Aftermath-The-(2019)',
 'Aliens-Ate-My-Homework',
 'All-Nighter',
 'All-Roads-Lead-to-Rome',
 'All-This-Panic',
 'Allure',
 'Allure-(Canada-2018)',
 'Along-the-Roadside',
 'American-Ascent-An',
 'American-Socialist-The-Life-and-Times-of-Eugene-Victor-Debs',
 'Amour-fou',
 'An-(Japan)',
 'Angel-of-Christmas',
 'Another-Barrio',
 'Apartment-212',
 'Apocalypsis',
 'Apollo-11-(Documentary)-(2019)',
 'Apprentice-(2016)',
 'Ashram-The',
 'Ask-Me-Anything',
 'Auge-Del-Humano-El-(Argentina)',
 'Automatic-Hate-The',
 'Avas-Possessions',
 'Avril-et-le-monde-truque',
 'Baaghi-2-(India)',
 'Baal-(Germany)',
 'Babylon-(UK)-(1980)',
 'Backcountry',

In [11]:
prefix = "https://www.the-numbers.com/graphs/movie/box-office-iframe/"
urls = list(map(lambda x: prefix +x, movies))
len(urls)

500

In [12]:
url = "https://www.the-numbers.com/graphs/movie/box-office-iframe/Star-Wars-Ep-VII-The-Force-Awakens"

In [13]:
def scrape_movie_data_manually(url: str) -> pd.DataFrame:
    """
    Pull Movie Earnings Data from url and pulling them into a DataFrame with Date, Revenue (USD)
    """
    pattern = r'data\.addRows\(([\s\S]*?)\);'
    response = requests.get(url)
    
    # Transform data
    parsed_data = re.search(pattern, response.text).group(1)
    cleaned_data = parsed_data.replace('\t', '').replace('\n', '').replace('\r', '').replace('new Date', '')
    movie_data = eval(cleaned_data)
    box_office_earnings = [
        (convert_date(*date), earnings)
        for date, earnings, *_ in movie_data
        if convert_date(*date) is not None
    ]
    
    return pd.DataFrame(box_office_earnings, columns=["Date", "Revenue"])

In [16]:
df_dict = {}

for i, movie in enumerate(movies):
    if i % 20 == 0:
        time.sleep(1)
        print(i)
    
    url = prefix + movie
    
    df_dict[movie] = scrape_movie_data_manually(url)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480


In [78]:
movie_data_filled = {x: y for x, y in df_dict.items() if len(y) > 0}

In [53]:
INFLATION_RATE = 0.021

def inf_adj_rev(df):
    release_year = df.iloc[0].Date.year
    rev = df.Revenue.max()
    return rev * (1 + INFLATION_RATE) ** (dt.date.today().year - release_year)

In [61]:
# Question: What is the most profitable movie that came out in the last 5 years that came out in March

sorted_gross_rev = pd.DataFrame(sorted(
    (
        (inf_adj_rev(movie_data), movie_data.Revenue.max(), title) 
        for title, movie_data in df_dict.items() 
        if len(movie_data) != 0
    ),
    reverse=True,
))
sorted_gross_rev.head()

,0,1,2
0,5.254050e+08,504014165.0,Beauty-and-the-Beast-(2017)
1,4.268298e+08,426829839.0,Captain-Marvel-(2019)
2,3.632228e+08,341268248.0,Zootopia-(2016)
3,3.516130e+08,330360194.0,Batman-v-Superman-Dawn-of-Justice
4,2.358805e+08,226277068.0,Logan-(2017)


In [65]:
from bokeh.palettes import Category20
from itertools import cycle

In [66]:
COLORS = cycle(Category20[20])

In [82]:
from collections import defaultdict

In [85]:
year_bucket = defaultdict(list)

for title, rev_df in sorted(movie_data_filled.items(), key=lambda x: x[1].Revenue.max(), reverse=True):
    release_year = rev_df.iloc[0].Date.year
    year_bucket[release_year].append((title, rev_df))

In [91]:
TOOLTIPS = [
    ("name", "$name"),
    ("(x,y)", "($x, $y)"),
]

for year, year_movie_data in sorted(year_bucket.items(), key=lambda x: x[0]):
    color_cycle = cycle(Category20[20])
    
    
    res = figure(
        width=800, 
        height=500, 
        tooltips=TOOLTIPS,
        x_axis_type='datetime', 
        title=f'{year} Revenue',
    )

    for (title, df), color in zip(year_movie_data, color_cycle):
        data = ColumnDataSource(df)
        res.line(x='Date', y='Revenue', source=data, line_color=color, name=title)

    show(res)

In [93]:
TOOLTIPS = [
    ("name", "$name"),
    ("(x,y)", "($x, $y)"),
]

for year, year_movie_data in sorted(year_bucket.items(), key=lambda x: x[0]):
    color_cycle = cycle(Category20[20])
    
    res = figure(
        width=800, 
        height=500, 
        tooltips=TOOLTIPS,
        x_axis_type='datetime', 
        title=f'{year} Revenue',
        y_axis_type="log",
    )

    for (title, df), color in zip(year_movie_data, color_cycle):
        data = ColumnDataSource(df)
        res.line(x='Date', y='Revenue', source=data, line_color=color, name=title)

    show(res)